# Постановка задачи

 

Дана таблица по транзакциям клиента в банке.

Дата. Описание. Сумма.

Пример.

 

Дата. Описание. Сумма.

01.02.20, Покупка телевизора, -50

01.04.20, Оплата связи, -10

01.01.20, Пополнение счета, +100

 

*в одну дату не может быть более одной транзакции

 

Вывести выписку по счету, упорядоченную по дате и с накопленным сальдо(остатком):

 

Дата. Описание. Сумма. Остаток.

01.01.20, Пополнение счета, +100, 100

01.02.20, Покупка телевизора, -50, 50

01.04.20, Оплата связи, -10, 40

 

*остаток счета до первой транзакции считаем нулевым

Дополнительное условие – без использования оконных функций.

### Решение

Решение можно представить следующими SQL-запросами (без использования оконных функций):  

df - название нашей таблицы.  
date, description, sum - наименования столбцов  

1. С использованием перекрестного соединения:

**SELECT x.date, y.sum  
FROM df AS x CROSS JOIN df AS y   
WHERE x.date >= y.date  
GROUP BY x.date  
ORDER BY x.date**

-----------------------------------------------------------------------------

2. С использованием рекурсивного запроса:

**WITH RecursiveCTE AS (  
    SELECT date, sum, sum AS running_total  
    FROM df  
    WHERE date = (SELECT MIN(date) FROM df)  
    UNION ALL  
    SELECT d.date, d.sum, r.running_total + d.sum  
    FROM df d   
    JOIN RecursiveCTE r ON d.date = DATE_ADD(r.date, INTERVAL 1 DAY)  
)  
SELECT date, running_total  
FROM RecursiveCTE  
ORDER BY date**  

Продемонстрируем ниже с использованием PostgreSQL.

In [17]:
#создадим таблицу
import psycopg2
import keyring

# Данные для записи
data = [
    ('2020-02-01', 'Покупка телевизора', -50),
    ('2020-04-01', 'Оплата связи', -10),
    ('2020-01-01', 'Пополнение счета', 100)
]

# Параметры подключения к базе данных
dbname = "postgres"
user = "postgres"
password = keyring.get_password('sql', 'password')
host = "localhost"
port = 5432

try:
    # Подключение к базе данных
    conn = psycopg2.connect(f"dbname={dbname} user={user} password={password} host={host} port={port}")
    cur = conn.cursor()

    # Создание таблицы (если она не существует)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS Transactions (
            date DATE,
            description VARCHAR(255),
            sum INTEGER
        )
    """)

    # Вставка данных
    for row in data:
        cur.execute("INSERT INTO Transactions (date, description, sum) VALUES (%s, %s, %s)", row)

    # Сохранение изменений
    conn.commit()

    print("Таблица создана и данные успешно записаны!")

except (Exception, psycopg2.Error) as error:
    print(f"Ошибка при работе с PostgreSQL: {error}")

finally:
    if conn:
        cur.close()
        conn.close()
        print("Соединение с PostgreSQL закрыто")

Таблица создана и данные успешно записаны!
Соединение с PostgreSQL закрыто


In [18]:
import psycopg2
import pandas as pd

# параметры подключения к базе данных

try:
    conn = psycopg2.connect(f"dbname={dbname} user={user} password={password} host={host} port={port}")
    cur = conn.cursor()

    # Запрос 1: Перекрестное соединение
    cur.execute("""
        SELECT x.date, SUM(y.sum) AS running_total
        FROM Transactions AS x CROSS JOIN Transactions AS y
        WHERE x.date >= y.date
        GROUP BY x.date
        ORDER BY x.date;
    """)
    df_cross_join = pd.DataFrame(cur.fetchall(), columns=['Дата', 'Кумулятивная сумма (Cross Join)'])

    # Запрос 2: Рекурсивный запрос
    cur.execute("""
    WITH RECURSIVE RecursiveCTE AS (
        SELECT date, sum, sum AS running_total
        FROM Transactions
        WHERE date = (SELECT MIN(date) FROM Transactions)
        UNION ALL
        SELECT d.date, d.sum, r.running_total + d.sum
        FROM Transactions d
        JOIN RecursiveCTE r ON d.date = (SELECT MIN(date) FROM Transactions WHERE date > r.date)
    )
    SELECT date, running_total
    FROM RecursiveCTE
    ORDER BY date;
    """)
    df_recursive = pd.DataFrame(cur.fetchall(), columns=['Дата', 'Кумулятивная сумма (Recursive)'])

    # Отображение датафреймов
    print("Результат перекрестного соединения:")
    print(df_cross_join)
    print("\nРезультат рекурсивного запроса:")
    print(df_recursive) 

except (Exception, psycopg2.Error) as error:
    print(f"Ошибка при работе с PostgreSQL: {error}")

finally:
    if conn:
        cur.close()
        conn.close()
        print("Соединение с PostgreSQL закрыто")

Результат перекрестного соединения:
         Дата  Кумулятивная сумма (Cross Join)
0  2020-01-01                              100
1  2020-02-01                               50
2  2020-04-01                               40

Результат рекурсивного запроса:
         Дата  Кумулятивная сумма (Recursive)
0  2020-01-01                             100
1  2020-02-01                              50
2  2020-04-01                              40
Соединение с PostgreSQL закрыто
